# genki-4k 数据集 人脸表情检测

## 定义face_cutter：识别人脸并裁剪

In [38]:
def face_cutter(file,k):
    import dlib
    import numpy as np
    import cv2
    # dlib预测器
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor
    ('/Users/zhaoxuyan/Desktop/shape_predictor_68_face_landmarks.dat')
    # 读取图片路径
    img = cv2.imread(file,0)
    path_save = '/Users/zhaoxuyan/Desktop/genki-4k-cutted-num/'
    
    # dlib检测
    dets = detector(img, 1)
    
    try:
        if(len(dets) == 1):
            for m,d in enumerate(dets):
                # 计算矩阵大小
                pos_start = tuple([d.left(), d.top()])
                pos_end = tuple([d.right(), d.bottom()])

                print (k)
                # 计算矩形框大小
                height = d.bottom() - d.top()
                width = d.right() - d.left()

                # 根据人脸大小生成空的图像
                img_blank = np.zeros((height, width, 3), np.uint8)

                for i in range(height):
                    for j in range(width):
                        img_blank[i][j] = img[d.top() + i][d.left() + j]
                
                cv2.imwrite(path_save+str(k)+".jpg", img_blank)
        else:
            print ("未检测到人脸的图片：",file)
    except Exception:
        pass

In [9]:
import os  
path_read = "/Users/zhaoxuyan/Desktop/genki-4k"  #待读取的文件夹  
files=os.listdir(path_read)  
files.sort() #对读取的路径进行排序  
k=0
for file in files: 
    k=k+1
    face_cutter(os.path.join(path_read,file),k)

## HOG特征提取

In [6]:
import dlib
import numpy as np
import cv2

def hog(image):
    hog = cv2.HOGDescriptor()
    h = hog.compute(image)
    return h

In [4]:
#提取目录下所有图片,更改尺寸后保存到另一目录  
from PIL import Image  
import os.path  
import glob  
def convertjpg(jpgfile,outdir,width=128,height=128):  
    img=Image.open(jpgfile)  
    try:  
        new_img=img.resize((width,height),Image.BILINEAR)     
        new_img.save(os.path.join(outdir,os.path.basename(jpgfile)))  
    except Exception as e:  
        print(e)  
for jpgfile in glob.glob("/Users/zhaoxuyan/Desktop/genki-4k-cutted-num/*.jpg"):  
    convertjpg(jpgfile,"/Users/zhaoxuyan/Desktop/genki-4k-resize-num")  

## 得到data和target

In [7]:
X=[]
import os  
path="/Users/zhaoxuyan/Desktop/genki-4k-resize-num"  #待读取的文件夹  
files=os.listdir(path)  
files.sort(key= lambda x:int(x[:-4])) #对读取的路径进行排序  
for file in files:  
    im = cv2.imread(os.path.join(path,file))
    h = hog(im)
    X.append(h)

## 去掉了识别不出人脸的图像

In [4]:
y_1 =  [1 for x in range(1,2163)]  
y_0 = [0 for x in range(2163,3778)]
y = y_1 + y_0

In [6]:
B=[]
for i in range(0,len(X)):
    A=X[i].reshape(1,-1)
    B.append(A[0])

In [8]:
data = B
target = y

## 训练集验证集拆分

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
        data, target, test_size=0.25, random_state=0)

## sklearn SVM

In [10]:
from sklearn.svm import SVC
clf = SVC(kernel='linear')

## k折交叉验证

In [11]:
from sklearn.model_selection import cross_val_score
import time
time_start=time.time()
scores = cross_val_score(clf, X_train, y_train, cv=2)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
time_end=time.time()
print('交叉验证时间：',time_end-time_start)

Accuracy: 0.82 (+/- 0.01)
交叉验证时间： 194.83248925209045


> - 2折交叉验证分数：82(+/- 1)%
- 交叉验证时间： 194.83248925209045 s

## 训练

In [12]:
from sklearn import metrics

def train_and_evaluate(clf, X_train, X_test, y_train, y_test):
    
    clf.fit(X_train, y_train)

    time_start=time.time()
    print ("Accuracy on training set:")
    print (clf.score(X_train, y_train)) 
    time_end=time.time()
    print('模型训练时长：',time_end-time_start)
    
    print ("Accuracy on testing set:")
    print (clf.score(X_test, y_test))
    
    y_pred = clf.predict(X_test)
    
    print ("Classification Report:")
    print (metrics.classification_report(y_test, y_pred))
    print ("Confusion Matrix:")
    print (metrics.confusion_matrix(y_test, y_pred))

## 评价(precision  recall  f1-score)

In [13]:
train_and_evaluate(clf, X_train, X_test, y_train, y_test)

Accuracy on training set:
1.0
模型训练时长： 151.17432117462158
Accuracy on testing set:
0.8317460317460318
Classification Report:
             precision    recall  f1-score   support

          0       0.81      0.80      0.81       414
          1       0.85      0.85      0.85       531

avg / total       0.83      0.83      0.83       945

Confusion Matrix:
[[332  82]
 [ 77 454]]


> - 模型训练时间：151.17432117462158 s
- 训练准确率：100%
- 验证准确率：83%
- 精确率：83%
- 召回率：83%
- F1 score：83%

## 保存训练好的模型到本地

In [14]:
from sklearn.externals import joblib
os.chdir("/Users/zhaoxuyan/Desktop/genki-4k-model")
joblib.dump(clf, "train_model.m")

['train_model.m']

## 从本地调回模型

In [2]:
from sklearn.externals import joblib
clf = joblib.load("/Users/zhaoxuyan/Desktop/genki-4k-model/train_model.m")

## 准备预测

In [2]:
#提取目录下所有图片,更改尺寸后保存到另一目录  
from PIL import Image  
import os.path  
import glob  
def convertjpg(jpgfile,outdir,width=128,height=128):  
    img=Image.open(jpgfile)  
    try:  
        new_img=img.resize((width,height),Image.BILINEAR)     
        new_img.save(os.path.join(outdir,os.path.basename(jpgfile)))  
    except Exception as e:  
        print(e)  
for jpgfile in glob.glob("/Users/zhaoxuyan/Desktop/pred/*.jpg"):  
    convertjpg(jpgfile,"/Users/zhaoxuyan/Desktop/pred_resize")  

In [6]:
import cv2
X=[]
import os  
path="/Users/zhaoxuyan/Desktop/pred_resize"  #待读取的文件夹  
files=os.listdir(path)  
for file in files:  
    print (os.path.join(path,file))
    im = cv2.imread(os.path.join(path,file),0)
    h = hog(im)
    X.append(h)

/Users/zhaoxuyan/Desktop/pred_resize/no_smile.jpg
/Users/zhaoxuyan/Desktop/pred_resize/smile.jpg


> 预测2张图片：no_smile和smile

## 预处理新图片（自拍）

In [7]:
B=[]
for i in range(0,len(X)):
    A=X[i].reshape(1,-1)
    B.append(A[0])

In [8]:
pred_data = B
pred_data

[array([0.3214328 , 0.3214328 , 0.09034893, ..., 0.22501577, 0.33847058,
        0.33847058], dtype=float32),
 array([0.30610967, 0.23619246, 0.12861842, ..., 0.1217628 , 0.23348024,
        0.2570294 ], dtype=float32)]

## 输出预测结果

In [9]:
clf.predict(pred_data)

array([0, 1])

> 输出是0和1，即不笑和笑，结果正确

## 摄像头实时预测

In [ ]:
from sklearn.externals import joblib


def hog(image):
    hog = cv2.HOGDescriptor()
    h = hog.compute(image)
    return h


clf = joblib.load("/Users/zhaoxuyan/Desktop/genki-4k-model/train_model.m")

if __name__ == '__main__':
    import dlib  # 人脸识别的库dlib
    import numpy as np  # 数据处理的库numpy
    import cv2  # 图像处理的库OpenCv
    from PIL import Image

    capture = cv2.VideoCapture(0)
    size = (int(capture.get(cv2.CAP_PROP_FRAME_WIDTH)), int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'DIVX')  # 'x264' doesn't work
    out = cv2.VideoWriter('/Users/zhaoxuyan/Desktop/result.mp4', fourcc, 20, size, False)

    # dlib预测器
    detector = dlib.get_frontal_face_detector()

    # 创建cv2摄像头对象
    cap = cv2.VideoCapture(0)

    # cap.set(propId, value)
    # 设置视频参数，propId设置的视频参数，value设置的参数值
    cap.set(3, 480)

    # 截图screenshoot的计数器
    cnt_ss = 0

    # 人脸截图的计数器
    cnt_p = 0

    # 保存
    path_save = "F:/code/python/P_dlib_face_reco/data/get_from_camera/"

    # cap.isOpened（） 返回true/false 检查初始化是否成功
    while cap.isOpened():
        # cap.read()
        # 返回两个值：
        #    一个布尔值true/false，用来判断读取视频是否成功/是否到视频末尾
        #    图像对象，图像的三维矩阵q
        flag, im_rd = cap.read()
        # 每帧数据延时1ms，延时为0读取的是静态帧
        kk = cv2.waitKey(1)
        # 取灰度
        img_gray = cv2.cvtColor(im_rd, cv2.COLOR_RGB2GRAY)
        # 人脸数rects
        rects = detector(img_gray, 0)
        # print(len(rects))
        # 待会要写的字体
        font = cv2.FONT_HERSHEY_SIMPLEX

        if len(rects) != 0:
            # 检测到人脸
            # 矩形框
            for k, d in enumerate(rects):

                # 计算矩形大小
                # (x,y), (宽度width, 高度height)
                pos_start = tuple([d.left(), d.top()])
                pos_end = tuple([d.right(), d.bottom()])

                # 计算矩形框大小
                height = d.bottom() - d.top()
                width = d.right() - d.left()

                # 根据人脸大小生成空的图像
                cv2.rectangle(im_rd, tuple([d.left(), d.top()]), tuple([d.right(), d.bottom()]), (0, 255, 255), 2)
                im_blank = np.zeros((height, width, 3), np.uint8)

                for i in range(height):
                    for j in range(width):
                        im_blank[i][j] = im_rd[d.top() + i][d.left() + j]

                # 存储人脸图像文件
                cv2.imwrite("/Users/zhaoxuyan/Desktop/camera-resize/1.jpg", im_blank)
                img = Image.open("/Users/zhaoxuyan/Desktop/camera-resize/1.jpg")
                # resize
                new_img = img.resize((128, 128), Image.BILINEAR)
                new_img.save("/Users/zhaoxuyan/Desktop/camera-resize/1.jpg")

            # 开始调用clf进行实时预测
            im = cv2.imread("/Users/zhaoxuyan/Desktop/camera-resize/1.jpg", 0)
            # hog特征提取
            h = hog(im)
            A = h.reshape(1, -1)
            B = [A[0]]
            pred_data = B
            result = clf.predict(pred_data)
            e = result.tolist()
            print(e)

            # im_rd = cv2.putText(im_rd, "status:" + str(e[0]), (20, 300), font, 1, (0, 0, 255), 1, cv2.LINE_AA)
            im_rd = cv2.putText(im_rd, "status:" + str(e[0]), (20, 350), font, 10, (0, 0, 255), 4, cv2.LINE_AA)

        else:
            # 没有检测到人脸
            print("未检测到人脸！")

        # 按下q键退出
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        out.write(im_rd)
        # 窗口显示
        # cv2.namedWindow("camera", 0) # 如果需要摄像头窗口大小可调
        cv2.imshow("camera", im_rd)

    # 释放摄像头
    cap.release()

    # 删除建立的窗口
    cv2.destroyAllWindows()

> - 摄像头实时视频见result.mp4
- 由于训练集大多数图像都没戴眼镜，测试时戴眼镜准确率很低
- 解决方法：取下眼镜后就OK啦~